## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
from utils import read_file, get_fs_data

### Permit Data Engineering

In [1]:
# county permit data
dfPlacerPermit = read_file("data\PermitData_Placer_040124.csv")
dfCSLTPermit   = read_file("data\PermitData_CSLT_040224.csv")
dfElDoPermit   = read_file("data\PermitData_ElDorado_040124.csv")
# trpa permit table
permitTable = "https://maps.trpa.org/server/rest/services/Permit_Records/MapServer/1"
dfTRPAPermit = get_fs_data(permitTable)

NameError: name 'read_file' is not defined

In [ ]:


folder_path = 'F:\Research and Analysis\Local Jurisdiction MOU data collection\Placer MOU Files\Placer'
dfs = []

# Loop through the files in the folder and identify CSV files
for file_name in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file_name)
    
    # Read the CSV file into a DataFrame and append to the list
    df = pd.read_excel(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
print(final_df)

### Annual Report Data Engineering

#### Permit Data